In [2]:
import sys
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [7]:
import urllib

In [5]:
# murl = "mongodb://admin:<PASSWORD>@clusterXXX&retryWrites=true"
# Atlas -> connection -> connect your application -> standard connection string -> copy
murl = 'mongodb://admin:'+urllib.parse.quote('비밀번호')+'standard connection string'


client = MongoClient(murl, 27017)

wc_match = pd.read_csv('./data/wc_match.tsv', sep='\t')
team = pd.read_csv('./data/team.tsv', sep='\t')
players = pd.read_csv('./data/players.tsv', sep='\t')

In [6]:
players.stat_value = players.stat_value.apply(lambda x: pd.to_numeric(x.replace('%', '')) / 100 if '%' in x \
                                            else pd.to_numeric(x.replace(',', '')) if ',' in x \
                                            else pd.to_numeric(x.split('\'')[0]) * 60 + pd.to_numeric(x.split('\'')[1].split('\"')[0]) if '\'' in x \
                                            else pd.to_numeric(x.replace('\"', '')) if '\"' in x \
                                            else pd.to_numeric(x) if ord(x[0]) < 60\
                                            else x)

documents = []
for match in wc_match.iterrows():
    document = {}
    home_players = list((v.dropna().to_dict()) for k, v in players[(players.match_id == match[1]['id']) & (players.team_id == match[1]['home_team_id']) & (players.half == 'total') & \
        (~players.stat_type.isin(['POS', 'SUB', 'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4', 'DZONE5', 'TZONE1', 'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5']))]\
.pivot_table(values='stat_value', index=['num', 'name', 'club', 'position'], columns='stat_type', aggfunc='first')\
.reset_index().iterrows())
    away_players = list((v.dropna().to_dict()) for k, v in players[(players.match_id == match[1]['id']) & (players.team_id == match[1]['away_team_id']) & (players.half == 'total') & \
        (~players.stat_type.isin(['POS', 'SUB', 'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4', 'DZONE5', 'TZONE1', 'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5']))]\
.pivot_table(values='stat_value', index=['num', 'name', 'club', 'position'], columns='stat_type', aggfunc='first')\
.reset_index().iterrows())

    document['date'] = match[1]['match_date']    
    document['home_team'] = {
        'country': team[team.id == match[1]['home_team_id']].country.values[0], 
        'score': match[1]['home_team_score'],
        'players': home_players
    }
    document['away_team'] = {
        'country': team[team.id == match[1]['away_team_id']].country.values[0], 
        'score': match[1]['away_team_score'],
        'players': away_players
    }
    documents.append(document)

client.Football.wc_match.remove()
client.Football.wc_match.insert_many(documents)
print(client.Football.wc_match.count())

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


{'$clusterTime': {'clusterTime': Timestamp(1541679754, 64),
  'signature': {'hash': b'\x9c\x97\xf9\xb9\x8fW_\xe7\xdc{=\x8e\xb5_3\xda\x7f\x07u\x04',
   'keyId': 6605358658400288769}},
 'electionId': ObjectId('7fffffff0000000000000008'),
 'n': 64,
 'ok': 1.0,
 'opTime': {'t': 8, 'ts': Timestamp(1541679754, 64)},
 'operationTime': Timestamp(1541679754, 64)}

64


/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
